Word Piece Model 은 논문에 subword unit 을 만드는 python 코드가 적혀있습니다. 편의를 위해서 일부분을 수정하였으며, tokenize 함수를 따로 만들어 두었습니다. 

In [1]:
from collections import Counter
from collections import defaultdict

class WordPieceModel:
    def __init__(self, max_iters=10):
        self.max_iters = max_iters if max_iters > 0 else 10
        self.units = {}
        self.max_length = 0

    def train(self, sents):
        vocabs = self._sent_to_vocabs(sents)
        self.units = self._build_subword_units(vocabs)

    def _sent_to_vocabs(self, sents):
        vocabs = Counter((eojeol.replace('_', '') for sent in sents for eojeol in sent.split() if eojeol))
        return {' '.join(w) + ' _': c for w,c in vocabs.items() if w}

    def _build_subword_units(self, vocabs):
        def get_stats(vocabs):
            pairs = defaultdict(int)
            for word, freq in vocabs.items():
                symbols = word.split()
                for i in range(len(symbols)-1):
                    pairs[(symbols[i],symbols[i+1])] += freq
            return pairs

        def merge_vocab(pair, v_in):
            v_out = {}
            bigram = ' '.join(pair)
            replacer = ''.join(pair)
            for word, freq in v_in.items():
                w_out = word.replace(bigram, replacer)
                v_out[w_out] = freq
            return v_out

        for i in range(1, self.max_iters + 1):
            print('\riterating {} / {} ...'.format(i, self.max_iters), end='', flush=True)
            pairs = get_stats(vocabs)
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            vocabs = merge_vocab(best, vocabs)
        print('\riterating {} / {} was done'.format(i, self.max_iters))

        units = {}
        for word, freq in vocabs.items():
            for unit in word.split():
                units[unit] = units.get(unit, 0) + freq
        self.max_length = max((len(w) for w in units))
        return dict(sorted(units.items(), key=lambda x:-x[1])[:self.max_iters])

    def tokenize(self, s):
        return ' '.join([self._tokenize(w) for w in s.split()])

    def _tokenize(self, w):
        def initialize(w):
            w += '_'
            subwords = []
            n = len(w)
            for b in range(n):
                for e in range(b+1, min(n, self.max_length)+1):
                    subword = w[b:e]
                    if not subword in self.units:
                        continue
                    subwords.append((subword, b, e, e-b))
            return subwords

        def longest_match(subwords):
            matched = []
            subwords = sorted(subwords, key=lambda x:(-x[3], x[1]))
            while subwords:
                s, b, e, l = subwords.pop(0) # str, begin, end, length
                matched.append((s, b, e, l))
                removals = []
                for i, (_, b_, e_, _) in enumerate(subwords):
                    if (b_ < e and b < e_) or (b_ < e and e_ > b):
                        removals.append(i)
                for i in reversed(removals):
                    del subwords[i]
            return sorted(matched, key=lambda x:x[1])

        subwords = initialize(w)
        subwords = longest_match(subwords)
        subwords = ' '.join([s for s, _, _, _ in subwords])
        return subwords


vocab = {'l o w _' : 5,
         'l o w e r _' : 2,
         'n e w e s t _':6,
         'w i d e s t _':3
        }

encoder = WordPieceModel()
encoder.units = encoder._build_subword_units(vocab)
encoder.units

iterating 10 / 10 was done


{'_': 2,
 'd': 3,
 'e': 2,
 'est_': 3,
 'low': 2,
 'low_': 5,
 'newest_': 6,
 'r': 2,
 'wi': 3}

In [2]:
encoder._tokenize('lower')

'low e r _'

In [3]:
encoder.tokenize('lower newest')

'low e r _ newest_'

In [4]:
import sys
from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset'.format(dataset_dir))

from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

corpus_path = get_news_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True, num_sent = 1000)

soynlp=0.0.49
Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)


In [5]:
news_encoder = WordPieceModel(max_iters=1000)
news_encoder.train(corpus)

iterating 1000 / 1000 was done


In [6]:
# sort by length
sorted(news_encoder.units.items(), key=lambda x:-len(x[0]))[:20]

[('2014년_', 14),
 ('연합뉴스_', 165),
 ('현지시간_', 47),
 ('확장억제_', 31),
 ('2016_', 28),
 ('자료사진_', 27),
 ('설명했다_', 24),
 ('국방장관_', 24),
 ('관계자는_', 17),
 ('강조했다_', 17),
 ('알려졌다_', 14),
 ('대통령이_', 14),
 ('인근에서_', 12),
 ('것이라고_', 12),
 ('탄자니아_', 12),
 ('예정이다_', 11),
 ('것이라는_', 11),
 ('것으로_', 90),
 ('19일_', 80),
 ('말했다_', 58)]

In [8]:
# sort by frequency
sorted(news_encoder.units.items(), key=lambda x:-x[1])[:20]

[('_', 1184),
 ('을_', 578),
 ('이_', 547),
 ('에_', 505),
 ('의_', 489),
 ('한_', 366),
 ('를_', 334),
 ('은_', 314),
 ('는_', 290),
 ('고_', 286),
 ('이', 283),
 ('에서_', 276),
 ('가_', 260),
 ('했다_', 219),
 ('대', 215),
 ('지', 210),
 ('로_', 196),
 ('시', 176),
 ('사', 175),
 ('도_', 174)]

In [7]:
for i, sent in enumerate(corpus):
    if i > 10:
        break
    print('sent: {}'.format(sent))
    print('tokenized: {}'.format(news_encoder.tokenize(sent)), end='\n\n')

sent: 19
tokenized: 19 _

sent: 1990
tokenized: 19 9 0_

sent: 52 1 22
tokenized: 5 2_ 1_ 2 2_

sent: 오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스
tokenized: 오 패 산 터널_ 총격 전_ 용의자_ 검거 _ 서울_ 연합뉴스_ 경찰 _ 관계 자 들이_ 19일_ 오후_ 서울_ 강북 구_ 오 패 산_ 터널_ 인근에서_ 사 제_ 총 기를_ 발 사 해_ 경찰 을_ 살 해 한_ 용의자_ 성 모 씨를_ 검거 하고_ 있다_ 성씨는_ 검거 _ 당시_ 서 바 이 벌 _ 게 임 에서_ 쓰 는_ 방 탄 조 에_ 까지_ 착 용 한_ 상태 였다_ 독 자 제공_ 영 상_ 처_ 연합뉴스_

sent: 서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다
tokenized: 서울_ 연합뉴스_ 김 경_ 기자_ 사 제_ 총 기로_ 경찰 을_ 살 해 한_ 범 인_ 성 모_ 4 6_ 씨는_ 주 도 면 밀 했다_

sent: 경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다
tokenized: 경찰 에_ 따르면_ 성씨는_ 19일_ 오후_ 강북 경찰 서_ 인 근 _ 부 동 산_ 업 소_ 밖 에서_ 부 동 산 업 자_ 이 모_ 6 7_ 씨가_ 나 오 기를_ 기 다 렸다_ 이 씨 는_ 평 소 에도_ 말 다 을_ 자 주_ 한_ 것으로_ 알려졌다_

sent: 이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다
tokeni